In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.join(dc).drop_duplicates()

In [8]:
dl=df.join(lm)

In [9]:
len(dl)

13

Working df ready. Now infer location.

In [10]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [11]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [12]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [13]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [14]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,"Újabb medvét gázoltak el, ezúttal Kovászna meg...",0,2019-10-21,2019. október 21.,https://szekelyhon.ro/aktualis/ujabb-medvet-ga...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Kovászna megye', 'medve']","Újabb medvét gázoltak el, ezúttal Kovászna meg...",...,1,2,0,0,"['Ozsdola', 'Kovászna']",Ozsdola,Ozsdola,45.9805,26.263,"Ozsdola, 0"
1,"Érdeklődik a medve a füstölt sonka iránt, de a...",1,2019-10-21,2019. október 21.,https://szekelyhon.ro/aktualis/megtermett-medv...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Homoródalmás', 'medve']","Érdeklődik a medve a füstölt sonka iránt, de a...",...,1,4,0,0,Homoródalmás,Homoródalmás,Homoródalmás,46.2344,25.457,"Homoródalmás, 1"
2,Bocsával együtt tört be az udvarra az anyamedv...,2,2019-10-22,2019. október 22.,https://szekelyhon.ro/aktualis/bocsaval-egyutt...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Tusnádfürdő']",Bocsával együtt tört be az udvarra az anyamedve,...,1,2,0,0,Tusnádfürdő,Tusnádfürdő,Tusnádfürdő,46.1461,25.857,"Tusnádfürdő, 2"
3,Még a kormányválság is a garázdálkodó medvékne...,3,2019-10-22,2019. október 22.,https://szekelyhon.ro/aktualis/meg-a-kormanyva...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Hargita megye', 'medve']",Még a kormányválság is a garázdálkodó medvékne...,...,1,0,0,0,NaN,NaN,,,,"N/A, 3"
4,Nagykadácson és Felsőboldogfalván is medve mia...,4,2019-10-24,2019. október 24.,https://szekelyhon.ro/aktualis/nagykadacson-es...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Udvarhelyszék']",Nagykadácson és Felsőboldogfalván is medve mia...,...,1,2,0,0,Felsőboldogfalva,Felsőboldogfalva,Felsőboldogfalva,46.2743,25.2775,"Felsőboldogfalva, 4"
5,"Nem sokat ér a medvebiztos kuka, ha mellé tesz...",5,2019-10-25,2019. október 25.,https://szekelyhon.ro/aktualis/nem-sokat-er-a-...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,['Zetelaka'],"Nem sokat ér a medvebiztos kuka, ha mellé tesz...",...,1,0,0,0,Zetelaka,Zetelaka,Zetelaka,46.391,25.3679,"Zetelaka, 5"
6,Már most kellene számolni a medvékkel a Marosv...,6,2019-10-26,2019. október 26.,https://szekelyhon.ro/aktualis/mar-most-kellen...,https://media.szekelyhon.ro/pictures/vasarhely...,aktualis,vasarhely,"['Közlekedés', 'Környezetvédelem', 'Marosvásár...",Már most kellene számolni a medvékkel a Marosv...,...,1,0,0,0,Marosvásárhely,Marosvásárhely,Marosvásárhely,46.5386,24.5514,"Marosvásárhely, 6"
7,Medgyes utcáiról zavarták el a medvétA csendőr...,7,2019-10-26,2019. október 26.,https://szekelyhon.ro/vilag/medgyes-utcairol-z...,https://media.szekelyhon.ro/pictures/csik/aktu...,vilag,csik,"['Belföld', 'medve']",Medgyes utcáiról zavarták el a medvét,...,1,2,0,0,NaN,NaN,,,,"N/A, 7"
8,Lépcsőházban járt a medveEgy fogarasi tömbház ...,8,2019-10-28,2019. október 28.,https://szekelyhon.ro/vilag/lepcsohazban-jart-...,https://media.szekelyhon.ro/pictures/csik/aktu...,vilag,csik,['Belföld'],Lépcsőházban járt a medve,...,1,2,0,0,"['Fogaras', 'Brassó']","['Fogaras', 'Brassó']",Fogaras,45.8416,24.9731,"Fogaras, 8"
9,Lépcsőházban járt a medveEgy fogarasi tömbház ...,8,2019-10-28,2019. október 28.,https://szekelyhon.ro/vilag/lepcsohazban-jart-...,https://media.szekelyhon.ro/pictures/csik/aktu...,vilag,csik,['Belföld'],Lépcsőházban járt a medve,...,1,2,0,0,"['Fogaras', 'Brassó']","['Fogaras', 'Brassó']",Brassó,45.658,25.6012,"Brassó, 8"


Save data for loading in viz app

In [15]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [16]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [17]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')